In [11]:
import pickle

with open("./out/ts-train-merges.txt", "rb") as f:
  merges = pickle.load(f)

with open("./out/ts-train-vocab.json", "rb") as f:
  vocab = pickle.load(f)

print(merges[:10])
print(list(vocab.items())[:10])

longest_vocab_token = sorted(list(vocab.values()), key=lambda x: len(x), reverse=True)[0]
print(longest_vocab_token)

[(b' ', b't'), (b'h', b'e'), (b' ', b'a'), (b' ', b's'), (b' ', b'w'), (b'n', b'd'), (b' t', b'he'), (b'e', b'd'), (b' ', b'b'), (b' t', b'o')]
[(0, b'<|endoftext|>'), (1, b'\x00'), (2, b'\x01'), (3, b'\x02'), (4, b'\x03'), (5, b'\x04'), (6, b'\x05'), (7, b'\x06'), (8, b'\x07'), (9, b'\x08')]
b' accomplishment'


In [15]:
for merge in merges:
  print(merge)

(b' ', b't')
(b'h', b'e')
(b' ', b'a')
(b' ', b's')
(b' ', b'w')
(b'n', b'd')
(b' t', b'he')
(b'e', b'd')
(b' ', b'b')
(b' t', b'o')
(b' a', b'nd')
(b' ', b'h')
(b' ', b'f')
(b'i', b'n')
(b' ', b'T')
(b' w', b'a')
(b'r', b'e')
(b'i', b't')
(b'o', b'u')
(b' ', b'l')
(b' ', b'd')
(b' ', b'c')
(b' ', b'p')
(b'a', b'y')
(b' ', b'm')
(b'e', b'r')
(b' wa', b's')
(b'o', b'm')
(b' T', b'he')
(b' ', b'he')
(b'i', b's')
(b'a', b'r')
(b' ', b'n')
(b'i', b'm')
(b'o', b'n')
(b' s', b'a')
(b'i', b'd')
(b'l', b'l')
(b' h', b'a')
(b' ', b'g')
(b'a', b't')
(b' ', b'S')
(b'in', b'g')
(b'o', b't')
(b'e', b'n')
(b'a', b'n')
(b'l', b'e')
(b'o', b'r')
(b'i', b'r')
(b'a', b'm')
(b'e', b't')
(b' ', b'H')
(b' ', b'it')
(b' t', b'h')
(b'i', b'g')
(b' The', b'y')
(b' ', b'in')
(b'i', b'l')
(b' p', b'l')
(b' H', b'e')
(b' ', b'"')
(b'o', b'w')
(b'v', b'er')
(b'r', b'i')
(b'u', b't')
(b' b', b'e')
(b' ', b'u')
(b' sa', b'id')
(b' pl', b'ay')
(b'it', b'h')
(b' d', b'ay')
(b' w', b'ith')
(b'p', b'p')
(b'O', b'n')
(b

In [16]:
for key, token in vocab.items():
  print(token, key)

b'<|endoftext|>' 0
b'\x00' 1
b'\x01' 2
b'\x02' 3
b'\x03' 4
b'\x04' 5
b'\x05' 6
b'\x06' 7
b'\x07' 8
b'\x08' 9
b'\t' 10
b'\n' 11
b'\x0b' 12
b'\x0c' 13
b'\r' 14
b'\x0e' 15
b'\x0f' 16
b'\x10' 17
b'\x11' 18
b'\x12' 19
b'\x13' 20
b'\x14' 21
b'\x15' 22
b'\x16' 23
b'\x17' 24
b'\x18' 25
b'\x19' 26
b'\x1a' 27
b'\x1b' 28
b'\x1c' 29
b'\x1d' 30
b'\x1e' 31
b'\x1f' 32
b' ' 33
b'!' 34
b'"' 35
b'#' 36
b'$' 37
b'%' 38
b'&' 39
b"'" 40
b'(' 41
b')' 42
b'*' 43
b'+' 44
b',' 45
b'-' 46
b'.' 47
b'/' 48
b'0' 49
b'1' 50
b'2' 51
b'3' 52
b'4' 53
b'5' 54
b'6' 55
b'7' 56
b'8' 57
b'9' 58
b':' 59
b';' 60
b'<' 61
b'=' 62
b'>' 63
b'?' 64
b'@' 65
b'A' 66
b'B' 67
b'C' 68
b'D' 69
b'E' 70
b'F' 71
b'G' 72
b'H' 73
b'I' 74
b'J' 75
b'K' 76
b'L' 77
b'M' 78
b'N' 79
b'O' 80
b'P' 81
b'Q' 82
b'R' 83
b'S' 84
b'T' 85
b'U' 86
b'V' 87
b'W' 88
b'X' 89
b'Y' 90
b'Z' 91
b'[' 92
b'\\' 93
b']' 94
b'^' 95
b'_' 96
b'`' 97
b'a' 98
b'b' 99
b'c' 100
b'd' 101
b'e' 102
b'f' 103
b'g' 104
b'h' 105
b'i' 106
b'j' 107
b'k' 108
b'l' 109
b'm' 110
b'n' 111